This notebook parses the benchmark results.

In [1]:
from pathlib import Path
from typing import NamedTuple
from itertools import product

import pandas as pd

## Constants

In [2]:
DATA_DIR = Path("../data/results/")
PROBLEMS = [
    "JSP",
    "FJSP",
    "NPFSP",
    "NW-PFSP",
    "HFSP",
    "PFSP",
    "SDST-PFSP",
    "TCT-PFSP",
    "TT-PFSP",
    "RCPSP",
    "MMRCPSP",
    "RCMPSP"
]
SOLVERS = ["ortools", "cpoptimizer"]
TIME_LIMITS = [120, 900]

## Helpers

In [3]:
class Result(NamedTuple):
    instance: str
    status: str
    objective: float
    time: float

def parse_results(loc: Path) -> list[Result]:
    with open(loc, "r") as fh:
        text = fh.read()

    text = text[text.index("Instance"):] # start at results header
    lines = [line.strip() for line in text.split("\n")]
    lines = lines[2:] # skip header
    lines = iter(lines)

    results = []
    while (line := next(lines)): # continue until next empty line
        instance, status, objective, time = line.split()
        result = Result(instance, status, float(objective), float(time))
        results.append(result)

    return results

## Parsing

In [4]:
class Experiment(NamedTuple):
    problem: str
    solver: str
    time_limit: int
    instance: str
    status: str
    objective: float
    time: float

data = [] 
for problem, solver, time_limit in product(PROBLEMS, SOLVERS, TIME_LIMITS):
    try:
        loc = DATA_DIR / problem / solver / str(time_limit) / "results.txt"
        results = parse_results(loc)
        
        for result in results:
            data.append(Experiment(problem, solver, time_limit, *result))
    except:
        print(loc)

../data/results/TCT-PFSP/ortools/900/results.txt


In [5]:
df = pd.DataFrame(data)
df.head()

,problem,solver,time_limit,instance,status,objective,time
0,JSP,ortools,120,1.txt,Optimal,1231.0,5.57
1,JSP,ortools,120,10.txt,Optimal,1241.0,25.23
2,JSP,ortools,120,100.txt,Feasible,3916.0,120.06
3,JSP,ortools,120,101.txt,Feasible,4642.0,120.04
4,JSP,ortools,120,102.txt,Optimal,4725.0,83.54


In [6]:
df.to_csv("../data/results.csv", index=False)